# 数据分析实战

### 1. 读取数据

    1. 先观察数据是否存在 “列名” 和 “index”

    2. 选择csv或者Excel方式读取（尽量选择read_csv方法，可以减少内存消耗）

In [ ]:
import pandas as pd  
import numpy as np  

data = pd.read_csv('AIDS_Classification.csv')

In [ ]:
data.head()

### 2. 检查是否存在空缺值和异常值

    1. 检测是否存在空缺值：isna()、isnull()方法 或者 missingno 库

In [ ]:
# 1. isnull()方法 -- 检查每列的空缺值数量
missing_values = data.isnull().sum()
print(missing_values)

In [ ]:
# 2. isnull()方法 -- 检查每列的空缺值数量
missing_values = data.isna().sum()
print(missing_values)

In [ ]:
# 3. missingno库
import missingno as msno  

msno.matrix(data)
# msno.bar(data)

    2. 检查是否存在异常值：箱型图 或者 3σ 原则

In [ ]:
# 1. 箱型图方法
import matplotlib.pyplot as plt 

# 设置图像大小
plt.figure(figsize=(20, 12))

# 绘制箱型图
data.boxplot()

# 添加图表标题和轴标签
plt.title('Box Plot of Variables')
plt.ylabel('Values')

# 显示图形
plt.show()

In [ ]:
# 2. 3σ 原则
means = data.mean()
stds = data.std()

for column in data.columns:
    upper_bound = means[column] + 3 * stds[column]
    lower_bound = means[column] - 3 * stds[column]
    print(f"{column} - Lower bound: {lower_bound}, Upper bound: {upper_bound}")
    print("Outliers:")
    print(data[(data[column] < lower_bound) | (data[column] > upper_bound)][column])

### 3. 相关性分析 （特征选择）

**我们想知道哪些特征与标签的关系最大！**

    1. 相关性分析（pearson线性相关性分析 / spearman相关性分析），但是，在进行pearson线性相关性分析前，我们首先需要检验每一列特征是否服从正态分布（高斯分布），因为，只有满足正态分布，pearson线性相关性分析才是有有意义的！

In [ ]:
# 1. 数据分布检验（seaborn库可视化方法 —— 直方图）
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 设置图像大小
plt.figure(figsize=(10, 6))

# 绘制每个特征的直方图
for column in data.iloc[:, 0:4].columns:
    sns.histplot(data[column], kde=True, label=column)

# 添加图例
plt.legend(title='Variable')

# 添加图表标题和轴标签
plt.title('Histograms of Each Feature')
plt.xlabel('Value')
plt.ylabel('Frequency')

# 显示图形
plt.show()

In [ ]:
# 1. 正态分布检验（Q-Q图方法）
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

plt.figure(figsize=(6, 6))
sm.qqplot(data['time'], line='45', fit=True)
plt.title('Q-Q Plot')
plt.show()

In [ ]:
#  1.正态分布检验（Shapro-Wilk检验 和 Kolmogorov-Smirnov检验）
#  S-W检验方法一般需要样本量小于50，如果样本量大于50建议使用K-S检验

import pandas as pd
import numpy as np
import scipy.stats as stats

# 对每列进行K-S检验
for column in data.columns:
    data_temp = data[column]
    kstest_result = stats.kstest(data_temp, 'norm', args=(data_temp.mean(), data_temp.std()))
    print(f"Column {column}:")
    print(f"  K-S Statistic: {kstest_result.statistic:.4f}, P-value: {kstest_result.pvalue:.4f}\n")

In [ ]:
# 2. 进行spearman相关性分析
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np


# 计算Spearman相关系数矩阵
spearman_corr = data.corr(method='spearman')

# 设置热力图的大小
plt.figure(figsize=(20, 12))

# 绘制热力图
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)

# 添加标题
plt.title('Spearman Correlation Heatmap')

# 显示图形
plt.show()

    2. 使用逻辑回归的方法分析相关性

In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression

# 分离特征和标签
X = data.drop(columns='infected')
y = data['infected']

# 实例化逻辑回归模型
model = LogisticRegression()

# 拟合模型
model.fit(X, y)

# 获取系数并创建系数DataFrame
coefficients = pd.DataFrame(model.coef_[0], index=X.columns, columns=['Coefficient'])

# 计算系数的绝对值并排序
coefficients['Absolute Value'] = coefficients['Coefficient'].abs()
coefficients = coefficients.sort_values(by='Absolute Value', ascending=False)

print(coefficients)

    3. 使用随机森林的方法分析相关性

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier


import matplotlib.pyplot as plt

# 分离特征和标签
X = data.drop(columns='infected')
y = data['infected']

# 实例化随机森林模型
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# 拟合模型
rf.fit(X, y)

# 获取特征重要性
feature_importances = rf.feature_importances_

# 创建特征重要性DataFrame并排序
features_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# 可视化特征重要性
plt.figure(figsize=(16, 6))
plt.bar(features_df['Feature'], features_df['Importance'], color='b')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.show()

**3种方法对比**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# 分离特征和标签
X = data.drop(columns='infected')
y = data['infected']


# 随机森林
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)
rf_importances = rf.feature_importances_

# 逻辑回归
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
log_reg = LogisticRegression()
log_reg.fit(X_scaled, y)
log_reg_coefs = np.abs(log_reg.coef_[0])

# Spearman相关性
spearman_corrs = np.abs(data.iloc[:, :-1].corrwith(data['infected'], method='spearman'))

# 创建结果DataFrame
result_df = pd.DataFrame({
    'RandomForest': pd.Series(rf_importances, index=X.columns),
    'LogisticRegression': pd.Series(log_reg_coefs, index=X.columns),
    'SpearmanCorrelation': spearman_corrs
})

# 排序每列
result_df = result_df.apply(lambda x: x.sort_values(ascending=False).index)
result_df = result_df.reset_index()
result_df.drop(columns='index',inplace=True)

**提取出3种方法共同的前10个中的特征**

In [ ]:
# 提取每种方法的前10个特征
top10_rf = result_df['RandomForest'].head(10)
top10_lr = result_df['LogisticRegression'].head(10)
top10_spearman = result_df['SpearmanCorrelation'].head(10)

# 计算三个列表的交集，找出共同的元素
common_features = set(top10_rf).intersection(set(top10_lr)).intersection(set(top10_spearman))

# 打印共同的特征
print("共同的特征有：", common_features)

### 4. 建立模型

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

# 假设data是已经加载的DataFrame，common_features是已经定义的共同特征列表
X_full = data.drop(columns='infected')
y = data['infected']
X_reduced = data[list(common_features)]

# 数据标准化
scaler_full = StandardScaler()
X_full_scaled = scaler_full.fit_transform(X_full)

scaler_reduced = StandardScaler()
X_reduced_scaled = scaler_reduced.fit_transform(X_reduced)

# 分割数据集
X_train_full, X_test_full, y_train, y_test = train_test_split(X_full_scaled, y, test_size=0.3, random_state=42)
X_train_red, X_test_red, _, _ = train_test_split(X_reduced_scaled, y, test_size=0.3, random_state=42)

# 定义模型
models = {
    'SVM': SVC(probability=True),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(max_depth=5),
    'Random Forest': RandomForestClassifier(n_estimators=1000),
    'Naive Bayes': GaussianNB(),
    'LightGBM': LGBMClassifier(max_depth=5,n_estimators=1000)
}

# 绘制每个模型的ROC曲线
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 15))
axes = axes.ravel()

for ax, (name, model) in zip(axes, models.items()):
    # 训练并预测完整特征集
    model.fit(X_train_full, y_train)
    preds_full = model.predict_proba(X_test_full)[:, 1]
    fpr_full, tpr_full, _ = roc_curve(y_test, preds_full)
    roc_auc_full = auc(fpr_full, tpr_full)
    
    # 训练并预测缩减特征集
    model.fit(X_train_red, y_train)
    preds_red = model.predict_proba(X_test_red)[:, 1]
    fpr_red, tpr_red, _ = roc_curve(y_test, preds_red)
    roc_auc_red = auc(fpr_red, tpr_red)
    
    # 绘制ROC曲线
    ax.plot(fpr_full, tpr_full, label=f'Full Features (AUC = {roc_auc_full:.2f})', color='blue')
    ax.plot(fpr_red, tpr_red, label=f'Common Features (AUC = {roc_auc_red:.2f})', color='red')
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curve for {name}')
    ax.legend(loc="lower right")

plt.tight_layout()
plt.show()

### 5. 数据标准化简介

    数据标准化是一种预处理技术，用于将数据调整到一个共同的规模，通常用于改善算法的性能和结果的可靠性。它特别重要对于那些依赖于数据尺度的机器学习算法，例如支持向量机（SVM）和最近邻（KNN），因为这些算法会基于数据点之间的距离来作出决策。

#### 常见的数据标准化方法

1. **最小-最大缩放（Min-Max Scaling）**：
   - 将数据缩放到一个指定的范围，通常是0到1，或者是-1到1。
   - 公式：
   $$ 
   X_{\text{norm}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}} 
   $$
   - 其中$ X_{\text{min}} $和$ X_{\text{max}} $分别是数据在该特征上的最小值和最大值。
   - 适应场景：当你需要数据严格限定在一定范围内时。

2. **Z得分标准化（Z-Score Normalization）**：
   - 也称为标准化或标准分数，它会将数据的均值调整为0，标准差调整为1。
   - 公式：
   $$ 
   X_{\text{standardized}} = \frac{X - \mu}{\sigma}
   $$
   - 其中$ \mu $是样本均值，$ \sigma $是样本标准差。
   - 适应场景：尤其适用于数据近似呈正态分布时。

3. **最大绝对值缩放（MaxAbs Scaling）**：
   - 将数据除以数据的最大绝对值以将每个特征缩放到[-1, 1]范围内，不移动/不居中数据。
   - 公式：
   $$
    X_{\text{scaled}} = \frac{X}{\max(\left|X\right|)}
   $$
   - 适应场景：处理稀疏数据时非常有用（不会破坏稀疏性）。

4. **单位长度缩放（Normalization to Unit Length）**：
   - 调整数据的尺度使得特征向量的欧氏长度（L2范数）为1。
   - 公式：
   $$ 
   X_{\text{normalized}} = \frac{X}{\left\| X \right\|_2}
   $$
   - 适应场景：有时用于文本分类和聚类，以及在使用点积计算相似度时。

#### 数据标准化的重要性

- **改进学习算法的收敛速度**：在梯度下降等优化算法中，如果一个特征的范围远大于另一个特征的范围，它可能会主导梯度下降的方向，使得学习过程变慢。
- **避免数值不稳定性**：大的数值范围可能导致数值计算上的不稳定，如溢出或下溢。
- **提高模型的准确度**：特别是对于基于距离的算法，如KNN、SVM，标准化可以防止某些特征因为尺度大而对距离计算产生过大影响。